# Question1

## data clean and vectorization 

#### create DataFrame

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

#data cleaning via RDD
lines = spark.read.text("/home/share/musicrate.txt").rdd
parts = lines.map(lambda row: row.value.split(";"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), musicId=int(p[1]),
                                     rating=float(p[2])))

In [4]:
ratings = spark.createDataFrame(ratingsRDD)
ratings.show()

+-------+------+------+
|musicId|rating|userId|
+-------+------+------+
|  76783|   7.0|   953|
|    907|   3.0|   954|
|  32555|   5.0|   954|
|  33010|   4.0|   954|
|   8078|   8.0|   954|
|  34709|  10.0|   954|
|   3494|   4.0|   954|
|   3448|   6.0|   954|
|   3453|  10.0|   954|
| 344015|   3.0|   954|
|  34882|   4.0|   954|
|   3418|   5.0|   954|
|  33629|   6.0|   954|
|  35333|   7.0|   954|
|   3171|   6.0|   954|
|   3185|   3.0|   954|
|   3502|   2.0|   954|
|   3214|   6.0|   954|
|  35085|   7.0|   954|
|   2542|   9.0|   954|
+-------+------+------+
only showing top 20 rows



In [6]:
(training, test) = ratings.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True,userCol="userId", itemCol="musicId", ratingCol="rating",
          coldStartStrategy="drop")

#fit training data into model
model = als.fit(training)

In [11]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
#rmse = evaluator.evaluate(predictions)
#print("Root-mean-square error = " + str(rmse))
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

In [15]:
print(1)

1


In [16]:
userRecs.show(3,False)

+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                         |
+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|65    |[[4773,6.349504], [29571,5.619185], [60911,4.6032057], [86970,4.6032057], [87851,4.6032057], [30187,4.6032057], [4627,3.9972641], [8395,3.74232], [70616,3.5844743], [90684,3.5583975]] |
|53    |[[8395,5.9981704], [4773,5.591066], [29571,5.019331], [94404,4.704097], [81450,4.116084], [7017,4.116084], [42996,4.02158], [26702,4.02158], [42589,4.02158], [6038,3.7646155]]         |
|155   |[[6044,3.3920922], [84

In [17]:
movieRecs.show(3,False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|musicId|recommendations                                                                                                                                                    |
+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|64590  |[[964,8.998238], [58,5.5098996], [963,5.0233936], [128,4.585006], [986,3.9047723], [157,3.6550195], [32,3.549104], [9,3.1741037], [56,3.08725], [48,3.034688]]     |
|96261  |[[964,7.9984326], [58,4.8976884], [963,4.4652386], [128,4.075561], [986,3.4709086], [157,3.248906], [32,3.1547594], [9,2.8214254], [56,2.7442222], [48,2.6975007]] |
|243    |[[157,2.9988499], [40,2.0716865], [32,2.0230718], [51,1.8751148], [123,1.5681485], [48,1.5671962], [52,1.5323517], [9,1.5

In [31]:
from pyspark.mllib.recommendation import Rating, ALS
Music = sc.textFile("/home/share/ml-100k/u.data")
print Music.first()
rawdata = Music.map(lambda x: x.split('\t'))
ratings = rawdata.map(lambda x: Rating(int(x[0]),int(x[1]),float(x[2])))
print ratings.first()
(training, test) = ratings.randomSplit([0.8, 0.2])
training.first()
test.first()

196	242	3	881250949
Rating(user=196, product=242, rating=3.0)


Rating(user=196, product=242, rating=3.0)

In [33]:
model = ALS.train(training, 50, 5, 0.01)
userFeatures = model.userFeatures()
movieFeatures = model.productFeatures()
print userFeatures.first()
print movieFeatures.first()

(1, array('d', [-0.27096691727638245, -0.1675601601600647, 0.07899501919746399, -0.46760717034339905, 0.5735558867454529, -0.007676607929170132, -0.27905896306037903, -0.21912045776844025, 0.3271646797657013, 0.31455808877944946, 0.23333097994327545, -0.25859037041664124, 0.06569837033748627, -0.5230371356010437, -0.22805385291576385, 0.030592046678066254, 0.4542900323867798, -0.07986529916524887, -0.4291997253894806, -0.2339172214269638, 0.10900458693504333, -0.44597259163856506, -0.35188573598861694, 0.23993417620658875, -0.1941814124584198, 0.3616299033164978, 0.3770653009414673, -0.11329542100429535, -0.5014241933822632, 0.07350672036409378, -0.226834237575531, 0.15342538058757782, 0.37904873490333557, -0.1535542607307434, -0.19556409120559692, 0.1466103196144104, -0.11856678873300552, -0.19386553764343262, 0.537994384765625, 0.15694577991962433, 0.5005782842636108, -0.12382523715496063, -0.06841724365949631, 0.021241948008537292, -0.36090102791786194, -0.053855519741773605, 0.2296

In [35]:
preRating = model.predict(196,242)
print preRating

3.76212127174


In [5]:
print 'recommend 5 movies to user666：'
reccomendmovie = model.recommendProducts(666,5)
for i in reccomendmovie:
    print i

recommend 5 movies to user666：
Rating(user=666, product=481, rating=5.575768521731508)
Rating(user=666, product=887, rating=5.430141864672151)
Rating(user=666, product=512, rating=5.30155208979875)
Rating(user=666, product=503, rating=5.266556326039051)
Rating(user=666, product=675, rating=5.135932534764814)


In [ ]:
movies666 = ratings.groupBy(lambda x : x.user).mapValues(list).lookup(666)
print 'the movies and its ratings that user666 have seen：'
for i in sorted(movies666[0],key=lambda x : x.rating,reverse=True):
    print ('%d     %f'%(i.product,i.rating))

In [6]:
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.evaluation import RankingMetrics
userMovies = ratings.map(lambda rating:(rating.user,rating.product))
predictions = model.predictAll(userMovies).map(lambda rating:((rating.user,rating.product), rating.rating))
ratingsAndPredictions = ratings.map(lambda rating:((rating.user,rating.product),rating.rating)).join(predictions)
predictedAndTrue = ratingsAndPredictions.map(lambda ((userId,product),(predicted, actual))
      :(predicted,actual))
regressionMetrics = RegressionMetrics(predictedAndTrue)
print ("Mean Squared Error : %f"%regressionMetrics.meanSquaredError)

Mean Squared Error : 0.084207


In [18]:
print predictedAndTrue.take(3)

[(3.0, 3.027571662864616), (2.0, 1.921936846348708), (4.0, 4.051456270669071)]


In [2]:
movie = spark.createDataFrame(movieRDD)
movie.show()
#ratings.select("userId","movieId","rating").filter(ratings.rating>2).show()

+-------+------+----------+------+
|movieId|rating| timestamp|userId|
+-------+------+----------+------+
|      2|   3.0|1424380312|     0|
|      3|   1.0|1424380312|     0|
|      5|   2.0|1424380312|     0|
|      9|   4.0|1424380312|     0|
|     11|   1.0|1424380312|     0|
|     12|   2.0|1424380312|     0|
|     15|   1.0|1424380312|     0|
|     17|   1.0|1424380312|     0|
|     19|   1.0|1424380312|     0|
|     21|   1.0|1424380312|     0|
|     23|   1.0|1424380312|     0|
|     26|   3.0|1424380312|     0|
|     27|   1.0|1424380312|     0|
|     28|   1.0|1424380312|     0|
|     29|   1.0|1424380312|     0|
|     30|   1.0|1424380312|     0|
|     31|   1.0|1424380312|     0|
|     34|   1.0|1424380312|     0|
|     37|   1.0|1424380312|     0|
|     41|   2.0|1424380312|     0|
+-------+------+----------+------+
only showing top 20 rows



#### splitting train/test datasets 

In [35]:
(training, test) = movie.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

## Build pipeline and model 

In [39]:
#start pipeline
#Build als model and set parameters
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True,userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

#fit training data into model
model = als.fit(training)



## Results  

In [38]:
# Evaluate the model by computing the RMSE(root mean square error) on the test data
#transform dataframe->model->dataframe
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

Root-mean-square error = 1.95357410039


In [40]:
userRecs.show(5,False)

+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                |
+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|28    |[[6,1.2390108], [54,1.2185225], [14,1.1332309], [3,1.1179535], [59,1.1144238], [34,1.0909408], [63,1.0676852], [20,1.0673373], [78,1.0354528], [65,1.017024]]  |
|26    |[[85,1.1621525], [58,1.1486164], [2,1.1011739], [88,1.0527321], [60,1.0263231], [0,1.0214522], [4,1.0199385], [44,1.0165259], [86,1.0112785], [21,1.0010533]]  |
|27    |[[51,1.1527448], [40,1.12884], [43,1.1161973], [44,1.0464054], [86,1.040949], [64,1.0365921], [18,1.0326712], [87,0.9890834], [21,0.98380446], [80,

In [132]:
from pyspark.mllib.regression import LabeledPoint


from pyspark.mllib.linalg import Vectors
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import NaiveBayes
pima = sc.textFile('/home/share/pima.txt')
pima_new = pima.map(lambda x: x.split(','))
pima_new.first()

[u'6', u'148', u'72', u'35', u'0', u'33.6', u'0.627', u'50', u'1']

In [133]:
label = pima_new.map(lambda x : x[-1])
data= pima_new.map(lambda x : (x[-1],x[0:-1])).map(lambda (x,y):(int(x), [float(i) for i in y])).map(lambda (x,y):LabeledPoint(x,Vectors.dense(y)))
data.take(4)

[LabeledPoint(1.0, [6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0]),
 LabeledPoint(0.0, [1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0]),
 LabeledPoint(1.0, [8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0]),
 LabeledPoint(0.0, [1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0])]

In [134]:
splits=data.randomSplit([0.8, 0.2])
train=splits[0]
test=splits[1]
print(train.count())
print(test.count())
print train.first()
print test.first()

625
143
(1.0,[6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0])
(1.0,[8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0])


In [135]:
LogReg = LogisticRegressionWithSGD.train(train, 15)
SVM = SVMWithSGD.train(train, 15)
nb = NaiveBayes.train(train)
print 'parameters of LogisticRegression：'
print(LogReg)
print 'parameters of SVM：'
print(SVM)
print 'parameters of naivebayes'
print (nb)

parameters of LogisticRegression：
(weights=[1.40059768505,11.6679310642,-10.6383455648,-2.89298096402,2.83201839661,-0.17941870617,0.0603824100052,0.508403278459], intercept=0.0)
parameters of SVM：
(weights=[1.39627142554,6.4036379116,-16.1864186146,-4.22649855505,-1.63036394269,-1.91952603871,0.0448259642593,-1.3995807259], intercept=0.0)
parameters of naivebayes


In [118]:
predictions = LogReg.predict(test.map(lambda x : x.features))
print 'prediction of class of first eight data：',predictions.take(10)
print 'real calss of first eight data：         ',test.map(lambda x : x.label).map(lambda x: int(x)).take(10)

prediction of class of first eight data： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
real calss of first eight data：          [1, 1, 0, 0, 0, 0, 0, 0, 1, 0]


In [114]:
lrTotalCorrect = test.map(lambda point : 1 if(lrModel.predict(point.features)==point.label) else 0).sum()
lrAccuracy = lrTotalCorrect/(test.count()*1.0)
print 'accuracy: %s'%lrAccuracy

accuracy: 0.660714285714


In [119]:
predictions = SVM.predict(test.map(lambda x : x.features))
print 'prediction of class of first eight data：',predictions.take(10)
print 'real calss of first eight data：         ',test.map(lambda x : x.label).map(lambda x: int(x)).take(10)

prediction of class of first eight data： [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
real calss of first eight data：          [1, 1, 0, 0, 0, 0, 0, 0, 1, 0]


In [136]:
predictions = nb.predict(test.map(lambda x : x.features))
print 'prediction of class of first eight data：',predictions.take(10)
print 'real calss of first eight data：         ',test.map(lambda x : x.label).map(lambda x: int(x)).take(10)

prediction of class of first eight data： [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
real calss of first eight data：          [1, 1, 1, 0, 0, 1, 0, 1, 0, 1]


In [122]:
LRacc = test.map(lambda point : 1 if(LogReg.predict(point.features)==point.label) else 0).sum()
SVMacc = test.map(lambda point : 1 if(SVM.predict(point.features)==point.label) else 0).sum()
LRacc = LRacc/(test.count()*1.0)
SVMacc = SVMacc/(test.count()*1.0)
print 'the accuracy of Logistic Regression: %f'%LRacc
print 'the accuracy of SVM: %f'%SVMacc

the accuracy of Logistic Regression: 0.660714
the accuracy of SVM: 0.666667


In [130]:
nbacc = test.map(lambda point : 1 if (nb.predict(point.features) == point.label) else 0).sum()
nbacc = nbacc/(1.0*test.count())
print 'the accuracy of Naivebayes: %f'%nbacc

the accuracy of Naivebayes: 0.690789


In [5]:
from pyspark import SparkContext   
data=sc.textFile("/home/share/wiki_00")  
import nltk  
from nltk.corpus import stopwords  
from functools import reduce  

In [6]:
text=data.collect()

In [7]:
from nltk.tokenize import word_tokenize
tokenized_docs = [word_tokenize(doc) for doc in text]
print repr(tokenized_docs[0]).decode("unicode-escape")

[u'{', u'``', u'id', u"''", u':', u'``', u'54663483', u"''", u',', u'``', u'url', u"''", u':', u'``', u'https', u':', u'//en.wikipedia.org/wiki', u'?', u'curid=54663483', u"''", u',', u'``', u'title', u"''", u':', u'``', u'Jonathan', u'M.', u'Lamb', u"''", u',', u'``', u'text', u"''", u':', u'``', u'Jonathan', u'M.', u'Lamb\n\nJonathan', u'Matthew', u'Lamb', u'(', u'born', u'November', u'8', u',', u'1981', u')', u'is', u'an', u'American', u'entrepreneur', u',', u'economist', u',', u'author', u',', u'and', u'speaker', u'.', u'\n\nBorn', u'in', u'Muncie', u',', u'Indiana', u',', u'Lamb', u'is', u'a', u'graduate', u'of', u'Ball', u'State', u'University', u'with', u'degrees', u'in', u'Economics', u',', u'and', u'Risk', u'Management', u',', u'and', u'has', u'an', u'MBA', u'from', u'North', u'Carolina', u'State', u'University.\n\nIn', u'2004', u',', u'Lamb', u'received', u'BS', u'degrees', u'from', u'the', u'Miller', u'College', u'of', u'Business', u'in', u'Risk', u'Management', u'and', u'Ec

In [8]:
import re
import string
regex = re.compile('[%s]' % re.escape(string.punctuation)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    

print repr(tokenized_docs_no_punctuation[2]).decode("unicode-escape")

[u'id', u'54663499', u'url', u'https', u'enwikipediaorgwiki', u'curid54663499', u'title', u'Online', u'discussion', u'platform', u'text', u'Online', u'discussion', u'platformnnAn', u'online', u'discussion', u'platform', u'is', u'an', u'online', u'platform', u'that', u'allows', u'for', u'or', u'is', u'build', u'specifically', u'for', u'online', u'discussion', u'nnIn', u'1979', u'students', u'from', u'Duke', u'University', u'created', u'the', u'first', u'online', u'discussion', u'platform', u'with', u'UsenetnnOnline', u'discussion', u'platforms', u'can', u'engage', u'people', u'in', u'collective', u'reflection', u'and', u'exchanging', u'perspectives', u'and', u'crosscultural', u'understandingnnPublic', u'display', u'of', u'ideas', u'can', u'encourage', u'intersubjective', u'meaning', u'makingnnOnline', u'discussion', u'platforms', u'may', u'be', u'an', u'important', u'structural', u'means', u'for', u'effective', u'largescale', u'participationnnOnline', u'discussion', u'platforms', u'can'

In [9]:
# Cleaning text of stopwords
from nltk.corpus import stopwords

tokenized_docs_no_stopwords = []

for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    
    tokenized_docs_no_stopwords.append(new_term_vector)

print repr(tokenized_docs_no_stopwords[20]).decode("unicode-escape")

[u'id', u'54663796', u'url', u'https', u'enwikipediaorgwiki', u'curid54663796', u'title', u'Kate', u'Manicom', u'text', u'Kate', u'ManicomnnKate', u'Zilpah', u'Manicom', u'11', u'March', u'1893', u'–', u'27', u'October', u'1937', u'British', u'suffragette', u'trade', u'unionistnnBorn', u'St', u'Pancras', u'area', u'London', u'Manicom', u'attended', u'Southfields', u'Girls', u'School', u'From', u'1911', u'active', u'women', u'suffrage', u'movement', u'She', u'associated', u'Sylvia', u'Pankhurst', u'worked', u'East', u'London', u'Federation', u'Suffragettes', u'She', u'also', u'joined', u'Labour', u'Party', u'Workers', u'Union', u'worked', u'organiser', u'1917', u'In', u'position', u'recruited', u'women', u'workers', u'across', u'country', u'played', u'key', u'role', u'strike', u'Pearl', u'Assurance', u'workersnnThe', u'Representation', u'People', u'Act', u'1918', u'enfranchised', u'women', u'thirty', u'Manicom', u'leading', u'campaigner', u'extend', u'franchise', u'women', u'basis', u'a

In [10]:
# Stemming and Lemmatizing
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []

for doc in tokenized_docs_no_stopwords:
    final_doc = []
    for word in doc:
        final_doc.append(porter.stem(word))
        #final_doc.append(snowball.stem(word))
        #final_doc.append(wordnet.lemmatize(word))
    
    preprocessed_docs.append(final_doc)

print repr(preprocessed_docs[20]).decode("unicode-escape")

[u'id', u'54663796', u'url', u'http', u'enwikipediaorgwiki', u'curid54663796', u'titl', u'Kate', u'Manicom', u'text', u'Kate', u'ManicomnnK', u'Zilpah', u'Manicom', u'11', u'March', u'1893', u'–', u'27', u'Octob', u'1937', u'British', u'suffragett', u'trade', u'unionistnnBorn', u'St', u'Pancra', u'area', u'London', u'Manicom', u'attend', u'Southfield', u'Girl', u'School', u'From', u'1911', u'activ', u'women', u'suffrag', u'movement', u'She', u'associ', u'Sylvia', u'Pankhurst', u'work', u'East', u'London', u'Feder', u'Suffragett', u'She', u'also', u'join', u'Labour', u'Parti', u'Worker', u'Union', u'work', u'organis', u'1917', u'In', u'posit', u'recruit', u'women', u'worker', u'across', u'countri', u'play', u'key', u'role', u'strike', u'Pearl', u'Assur', u'workersnnTh', u'Represent', u'Peopl', u'Act', u'1918', u'enfranchis', u'women', u'thirti', u'Manicom', u'lead', u'campaign', u'extend', u'franchis', u'women', u'basi', u'appli', u'men', u'When', u'Worker', u'Union', u'becam', u'part',

In [11]:
len(preprocessed_docs)

893

In [12]:
pre_svm=[]
for i in range(len(preprocessed_docs)):
    string = ' '.join(preprocessed_docs[i])
    pre_svm.append(string)
print repr(pre_svm[89]).decode("unicode-escape")

u'id 54664620 url http enwikipediaorgwiki curid54664620 titl Charl A OReilli III text Charl A OReilli IIInnCharl A OReilli III American academ He Frank E Buck Professor Manag Stanford Graduat School Busi He coauthor three book number case studi well coeditor fourth booknnCharl A OReilli III graduat Univers Texa El Paso earn bachelor scienc degre chemistri 1965 He earn MBA inform system 1971 PhD organiz behavior industri relat Haa School Busi Univers California BerkeleynnOReilli assist professor UCLA Anderson School Manag 1976 1978 He taught Haa School Busi 1979 1993 becam tenur professor Sinc 1993 Frank E Buck Professor Manag Stanford Graduat School BusinessnnOReilli coauthor three book coeditor fourth book He publish case studi Cisco System IBM Hi first book The Manag Organ Strategi Tactic Analys publish 1989 In 1997 publish second book Win Through Innov A Practic Guid Manag Organiz Chang Renew book highlight import teamwork foster innov Hi third book Hidden Valu How Compani Get Extra

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import dump_svmlight_file
import numpy as np
import os

#y = sentiments
#y = np.zeros(len(preprocessed_docs))
#i = 0

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(pre_svm)
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
#f = open('/home/share/tezheng.txt', 'w')
#dump_svmlight_file(tfidf, k, f, zero_based=False)
#f.close()
weight=tfidf.toarray()
#print X   
#print repr(vectorizer.get_feature_names()).decode("unicode-escape")
#print tfidf.toarray()
#print repr(tfidf.toarray()).decode("unicode-escape")
#print len(vectorizer.get_feature_names())

In [14]:
print weight.shape
count=0
for i in range(weight.shape[0]):
    for j in range(weight.shape[1]):
        if weight[i,j]==0 :
            count+=1
print count        

(893, 19714)
17534899


In [18]:
wssse = model.computeCost(sc.parallelize(weight))
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 876.01160769


In [16]:
from pyspark.mllib.clustering import KMeans, KMeansModel
model = KMeans.train(sc.parallelize(weight), 2, maxIterations=10, initializationMode="random")

In [1]:
data = spark.read.format("libsvm").load("/home/share/tezheng.txt")
data.show(20)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(19714,[557,641,7...|
|  1.0|(19714,[700,1025,...|
|  1.0|(19714,[551,689,1...|
|  1.0|(19714,[111,575,1...|
|  1.0|(19714,[621,689,6...|
|  1.0|(19714,[127,1029,...|
|  1.0|(19714,[26,119,70...|
|  1.0|(19714,[99,344,35...|
|  1.0|(19714,[456,545,7...|
|  1.0|(19714,[1033,2758...|
|  1.0|(19714,[99,560,71...|
|  1.0|(19714,[1035,2440...|
|  1.0|(19714,[278,281,2...|
|  0.0|(19714,[719,732,8...|
|  1.0|(19714,[1038,5523...|
|  1.0|(19714,[28,58,706...|
|  1.0|(19714,[55,68,85,...|
|  1.0|(19714,[1041,2483...|
|  1.0|(19714,[449,547,6...|
|  1.0|(19714,[530,544,5...|
+-----+--------------------+
only showing top 20 rows



In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
splits = data.randomSplit([0.9, 0.1])
train = splits[0]
test = splits[1]
lr = LogisticRegression(maxIter=15, regParam=0.4, elasticNetParam=0.8)

# train the model
model = lr.fit(train)

# select example rows to display.
predictions = model.transform(test)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(19714,[23,719,78...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[24,265,60...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[24,603,71...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[53,62,155...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[55,68,530...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[68,599,62...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[68,609,62...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[85,606,15...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[146,398,6...|[-0.9354051278974...|[0.28182941903584...|       1.0|
|  0.0|(19714,[3

In [111]:
k.shape

(893,)

In [56]:
''''
from pyspark.ml.clustering import KMeans
dataset = spark.read.format("libsvm").load("/home/share/tezheng.txt")
kmeans = KMeans().setK(10).setSeed(1)
model = kmeans.fit(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Within Set Sum of Squared Errors = 848.898676479
Cluster Centers: 
[  1.04551420e-04   9.34171539e-05   8.75270184e-05 ...,   1.46681863e-04
   2.05221421e-04   9.18971448e-05]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.  0.  0. ...,  0.  0.  0.]


In [57]:
prediction=model.predict(weight)

AttributeError: 'KMeansModel' object has no attribute 'predict'

In [49]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
df = spark.read.text("/home/share/tezheng.txt")
sentenceData = df.select("sentiment","text").toDF("label","text")

AnalysisException: u"cannot resolve '`sentiment`' given input columns: [value];;\n'Project ['sentiment, 'text]\n+- Relation[value#262] text\n"

In [50]:
wiki=sc.textFile('/home/share/tezheng.txt')

In [51]:
wiki.first()

u'0 558:0.06170331146465009 642:0.0546609463902866 720:0.02651654620043781 744:0.08252079862417232 1025:0.08252079862417232 2174:0.08252079862417232 2349:0.06077333840307329 2621:0.03695350515079095 3116:0.04926642988214861 3128:0.0576565237902821 3252:0.07446749527694212 3379:0.07780991815911048 3566:0.05520143736844081 3782:0.04995006592522476 3808:0.02801455714677534 3853:0.08252079862417232 4008:0.07187491067281829 4027:0.07187491067281829 4091:0.09920644088610379 4314:0.1300914686936369 4936:0.0396616972838975 5024:0.06382160732558809 5510:0.08252079862417232 6406:0.0385543396958262 6604:0.06796562118073285 6653:0.09168262595939893 7289:0.05317571937423406 7322:0.2186437855611464 7326:0.07780991815911048 7422:0.06504573434681844 7554:0.06170331146465009 7606:0.06641419192971193 7615:0.01161846080182349 7644:0.0576565237902821 8074:0.08252079862417232 8206:0.02942974229947002 8554:0.0541444853292854 8974:0.0460911819820552 9126:0.06641419192971193 9373:0.02925053369560278 9719:0.01